# Example 12 - Train Co-training

In this notebook, we'll set up the co-training process: (1) read cropped samples, get (npy, contextual features, label), (2) for each threshold: use train, conduct co-training, what is loss function? classify AH or not, use val to perform early stopping, select parameters, (3) each threshold has a co-training model, apply to test data, classifiation results, TP = classify (detect as P) as P, FP = classify (detect not P) as P, total P = number of P in annotations, recall = TP/#P, precision = TP/(TP + FP). 

In [1]:
import os
import glob
import numpy as np
import pickle
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler

import torchvision.models as models

from skorch import NeuralNetClassifier, NeuralNetBinaryClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from src.classifiers import CoTrainingClassifier

%matplotlib inline

## Step 1. Load train and val dataset

In this step, we'll load train and val dataset. Besides npy data, also need to extract contextual features, and labels. 

In [41]:
# load annotations
pkl_dir = "pkl/"
with open(pkl_dir + 'annotations_dict_new_p4.pickle', 'rb') as handle:
    annotations_dict = pickle.load(handle)

In [42]:
train_dir = "npy/train/"
val_dir = "npy/val/"

In [43]:
# threshold range
threshold_li = [-54, -56, -58, -60, -62, -64, -66, -68, -70, -72, -74, -76, -78, -80] 
threshold = -66

In [86]:
# get data for a certain threshold
# Note: X, y, all numpy arrays, in shape (n_samples, n_channels, width, height), (n_samples, )
def get_train_data(threshold):
    new_train_dir = train_dir + "threshold_" + str(threshold)
    new_val_dir = val_dir + "threshold_" + str(threshold)
    # merge
    # all_files = glob.glob(new_train_dir + '/*') + glob.glob(new_val_dir + '/*')
    all_files = glob.glob(new_train_dir + '/*')
    # export
    all_npy = []
    all_features = []
    all_labels = []
    print(len(all_files))
    pos_num = 0
    neg_num = 0
    unlabeled_num = 0
    for idx, file in enumerate(all_files):
        if idx % 10000 == 0:
            print(idx)
        # get contextual features
        features = (file.split('/')[3]).split('_')
        filename = features[0]
        features_dict = {'total_water_column': float(features[2]), 'depth': float(features[3]), 'relative_altitude': float(features[4]), 'latitude': float(features[5]), 'longitude': float(features[6]), 'time': features[7]}               
        # get label: 0 - neg, 1 - pos, -1 - unlabeled
        label = int(features[8].split('.')[0])
        if label == 4:
            label = 1
            pos_num += 1
        elif label == 0:
            # check, if no annotations on that file
            if filename in annotations_dict:
                label = -1
            else:
                label = 0 # get negative                
        else:
            label = -2
        # select, pos_num keeps increasing! how about small objects?
        if (label == 1) or (label == -2) or ((label == 0) and (neg_num <= pos_num * 5)) or ((label == -1) and (unlabeled_num <= (pos_num + neg_num) * 1)): 
            # get npy (time consuming)
            npy = np.load(file)
            npy = np.transpose(npy, (2, 0, 1))
            # add
            all_npy.append(npy)
            all_features.append(features_dict)   
            if label == -2:
                label = 0 # reassign
            all_labels.append(label)
            if label == 0:
                neg_num += 1
            if label == -1:
                unlabeled_num += 1
    return np.array(all_npy), all_features, np.array(all_labels)     

In [87]:
X1, X_feats_train, y = get_train_data(threshold)

256216
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000


In [88]:
### convert to X2, (n_samples, n_features)
feats_df = pd.DataFrame(X_feats_train)
sel_feats = ['depth', 'total_water_column', 'latitude', 'longitude']
feats_df_sel = feats_df[sel_feats] # select features
X2 = feats_df_sel.values
# impute nan
X2 = np.nan_to_num(X2) # some NaN in X2, impute use 0

In [89]:
# convert X1 to float
X1 = X1.astype(np.float32)

In [90]:
X1.dtype

dtype('float32')

In [91]:
# check shape, train: 256216, val: 90844
# labeled: 1240
# select process, keep labeled & unlabeled in 1:2
print(X1.shape)
print(X2.shape)
print(np.unique(y, return_counts=True))
print(y.shape)

(8601, 4, 100, 100)
(8601, 4)
(array([-1,  0,  1]), array([4301, 3581,  719]))
(8601,)


## Step 2. Define CNN model

In this step, we'll use skorch library to define CNN model, in particular, use ResNet18 model with 4 channels as input. The purpose is to wrap CNN model with different methods, fit, predict, predict_proba. 

In [92]:
torch.cuda.device_count()

2

In [93]:
torch.cuda.get_device_name(0), torch.cuda.get_device_name(1)

('NVIDIA GeForce GTX 1080 Ti', 'NVIDIA GeForce GTX 1080 Ti')

In [94]:
# define model
model_ft = models.resnet18(pretrained=False)
# change input to 4 channels
model_ft.conv1 = torch.nn.Conv2d(4, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False) # 4 channels
# define device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [95]:
print(torch.cuda.is_available())

True


In [96]:
torch.manual_seed(0)
cnn_1 = NeuralNetClassifier(model_ft, criterion=nn.CrossEntropyLoss(), max_epochs=20, lr=0.001, optimizer=optim.Adam, device=device)
# by default, 80/20 - train/val

## Step 3. Train & Test Co-training

In this step, we'll set up co-training using CNN and RF model. Each model uses different features, i.e., npy, and contextual features. 

In [97]:
co_clf = CoTrainingClassifier(cnn_1, RandomForestClassifier(), k=10, u=200)
co_clf.fit(X1, X2, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.1321       0.8012        0.6857  2.7065
      2        0.2339       0.8233        0.3427  2.1400
      3        0.1895       0.8337        0.2752  2.1144
      4        0.1642       0.8477        0.2596  2.1166
      5        0.1475       0.8442        0.3029  2.0980
      6        0.1412       0.8326        0.3850  2.0969
      7        0.1284       0.8744        0.2657  2.1100
      8        0.1334       0.8663        0.3309  2.1038
      9        0.1339       0.8651        0.3460  2.0963
     10        0.1225       0.8640        0.2719  2.0954
     11        0.1028       0.8512        0.4703  2.1079
     12        0.1292       0.8570        0.3463  2.1050
     13        0.0993       0.8674        0.3316  2.1287
     14        0.1127       0.8500        0.4278  2.1800
     15        0.1096       0.8744        0.3712  2.0988
     16        0.0940       0.8

      8        0.0001       0.8704        0.9711  2.1825
      9        0.0001       0.8704        0.9879  2.1844
     10        0.0001       0.8704        1.0027  2.1711
     11        0.0001       0.8704        1.0160  2.1792
     12        0.0001       0.8704        1.0283  2.1532
     13        0.0001       0.8704        1.0397  2.1503
     14        0.0000       0.8704        1.0503  2.1550
     15        0.0000       0.8704        1.0603  2.1496
     16        0.0000       0.8704        1.0697  2.1481
     17        0.0000       0.8716        1.0787  2.1635
     18        0.0000       0.8716        1.0872  2.1656
     19        0.0000       0.8716        1.0954  2.1603
     20        0.0000       0.8716        1.1032  2.1555
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.0100       0.8526        1.2223  2.1695
      2   

     16        0.0000       0.8778        0.9664  2.2158
     17        0.0000       0.8778        0.9752  2.2503
     18        0.0000       0.8778        0.9836  2.1610
     19        0.0000       0.8778        0.9917  2.1517
     20        0.0000       0.8778        0.9995  2.2506


In [98]:
# save model
pkl_dir = "pkl/"
with open(pkl_dir + "new_model_co_training.pkl", "wb") as f:
    pickle.dump(co_clf, f)

In [122]:
# load test data
def get_test_data(threshold):
    test_dir = "npy/val/"
    new_test_dir = test_dir + "threshold_" + str(threshold)
    all_files = glob.glob(new_test_dir + "/*")
    # export
    all_npy = []
    all_features = []
    all_labels = []
    print(len(all_files))
    for idx, file in enumerate(all_files):
        if idx % 10000 == 0:
            print(idx)
        # get contextual features
        features = (file.split('/')[3]).split('_')
        filename = features[0]
        features_dict = {'total_water_column': float(features[2]), 'depth': float(features[3]), 'relative_altitude': float(features[4]), 'latitude': float(features[5]), 'longitude': float(features[6]), 'time': features[7]}               
        # get label: 0 - neg, 1 - pos, -1 - unlabeled
        label = int(features[8].split('.')[0])       
        if label == 4:
            label = 1
        elif label == 0:
#             if filename in annotations_dict:
#                 label = -1
#             else:
#                 label = 0
              label = -1
        else:
            label = 0
        # select, only labeled data
        if label == 0 or label == 1: 
            # get npy
            npy = np.load(file)
            npy = np.transpose(npy, (2, 0, 1))
            all_npy.append(npy)
            all_features.append(features_dict)             
            all_labels.append(label)
    return np.array(all_npy), all_features, np.array(all_labels)     

In [123]:
X1_test, X_feats_test, y_test = get_test_data(threshold)

90844
0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [124]:
# convert to X2, (n_samples, n_features)
feats_df = pd.DataFrame(X_feats_test)
feats_df_sel = feats_df[sel_feats] # select features
X2_test = feats_df_sel.values

In [125]:
# convert X1 to float
X1_test = X1_test.astype(np.float32)

In [126]:
print(X1_test.shape)
print(X2_test.shape)
print(np.unique(y_test, return_counts=True))

(261, 4, 100, 100)
(261, 4)
(array([0, 1]), array([ 45, 216]))


In [127]:
c_y_predict = co_clf.predict(X1_test, X2_test)
CO_report = classification_report(y_test, c_y_predict, output_dict=True)
print(CO_report['1'])

{'precision': 0.9130434782608695, 'recall': 0.875, 'f1-score': 0.8936170212765957, 'support': 216}


## Step 4. Using only RF model

In this step, we'll use only RF model to do classification. See how it works!

In [105]:
# change -1 to 0
y_change = []
X2_change = []
for idx, item in enumerate(y):
    if item == -1:
        y_change.append(0)
    else:
        y_change.append(item)
    X2_change.append(X2[idx])

In [106]:
# remove -1
y_remove = []
X1_remove = []
X2_remove = []
for idx, item in enumerate(y):
    if item != -1:
        y_remove.append(item)
        X1_remove.append(X1[idx])
        X2_remove.append(X2[idx])

In [107]:
rf_clf = RandomForestClassifier() # LogisticRegression()
rf_clf.fit(np.array(X2_remove), np.array(y_remove))

RandomForestClassifier()

In [128]:
rf_predict = rf_clf.predict(X2_test)
RF_report = classification_report(y_test, rf_predict, output_dict=True)
print(RF_report['1'])

{'precision': 0.9624413145539906, 'recall': 0.9490740740740741, 'f1-score': 0.9557109557109558, 'support': 216}


## Step 5. Using only CNN model

In this step, we'll use only CNN model to do classification. See how it works!

In [109]:
# define model
model_ft = models.resnet18(pretrained=False)
# change input to 4 channels
model_ft.conv1 = torch.nn.Conv2d(4, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False) # 4 channels
# define device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [110]:
torch.manual_seed(1)
cnn_2 = NeuralNetClassifier(model_ft, criterion=nn.CrossEntropyLoss(), max_epochs=20, lr=0.001, optimizer=optim.Adam, device=device)
# by default, 80/20 - train/val

In [111]:
cnn_2.fit(np.array(X1_remove), np.array(y_remove))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.0751       0.6323        1.1918  2.1484
      2        0.2395       0.8202        0.3239  2.1805
      3        0.1983       0.8213        0.3127  2.1967
      4        0.1606       0.8499        0.2798  2.2211
      5        0.1468       0.8419        0.2991  2.2570
      6        0.1453       0.8247        0.4394  2.5735
      7        0.1338       0.8648        0.2934  2.1233
      8        0.1184       0.8545        0.2695  2.1638
      9        0.1034       0.8843        0.2425  2.1273
     10        0.0811       0.8259        0.4173  2.9050
     11        0.0903       0.8591        0.2829  2.6586
     12        0.0767       0.8328        0.4039  2.4278
     13        0.0769       0.8729        0.3181  2.2302
     14        0.0879       0.8522        0.4480  2.1236
     15        0.0736       0.8660        0.4069  2.1204
     16        0.0668       0.8

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ResNet(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(6

In [112]:
# save model
with open(pkl_dir + "new_model_cnn.pkl", "wb") as f:
    pickle.dump(cnn_2, f)

In [129]:
# test!
cnn_predict = cnn_2.predict(X1_test)
CNN_report = classification_report(y_test, cnn_predict, output_dict=True)
print(CNN_report['1'])

{'precision': 0.8761904761904762, 'recall': 0.8518518518518519, 'f1-score': 0.863849765258216, 'support': 216}
